In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pyodbc

# Thông tin kết nối PostgreSQL
postgres_config = {
    "host": "localhost",
    "database": "cafeland",
    "user": "postgres",
    "password": "123"
}

# Thông tin kết nối SQL Server
sql_server_config = {
    "server": "LAPTOP-OICT1D2U\\SQLEXPRESS",  # Tên server SQL Server
    "port": "1433",                # Cổng SQL Server (thường là 1433)
    "database": "cafeland",        # Tên database trong SQL Server
    "user": "sa",                  # Tài khoản SQL Server
    "password": "sapassword"       # Mật khẩu SQL Server
}

# Kết nối tới PostgreSQL
def connect_postgres(config):
    connection_string = f"postgresql+psycopg2://{config['user']}:{config['password']}@{config['host']}/{config['database']}"
    return create_engine(connection_string)

# Kết nối tới SQL Server
def connect_sql_server(config):
    connection_string = (
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={config['server']},{config['port']};"
        f"DATABASE={config['database']};"
        f"UID={config['user']};"
        f"PWD={config['password']};"
    )
    return pyodbc.connect(connection_string)

# Tạo bảng trong SQL Server nếu chưa tồn tại
def create_table_if_not_exists(cursor, table_name, df):
    # Tạo câu lệnh CREATE TABLE từ các cột của DataFrame
    columns_with_types = []
    for col in df.columns:
        # Giả sử rằng tất cả các cột là NVARCHAR, bạn có thể thay đổi kiểu dữ liệu nếu cần
        columns_with_types.append(f"[{col}] NVARCHAR(MAX)")
    
    columns_sql = ', '.join(columns_with_types)
    sql_query = f"IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{table_name}') " \
                f"BEGIN CREATE TABLE [{table_name}] ({columns_sql}) END"
    
    cursor.execute(sql_query)
    print(f"Bảng '{table_name}' đã được tạo hoặc đã tồn tại.")

# Lưu dữ liệu vào SQL Server
def write_to_sql_server(df, table_name, sql_conn):
    try:
        cursor = sql_conn.cursor()
        
        # Tạo bảng nếu chưa tồn tại
        create_table_if_not_exists(cursor, table_name, df)
        
        # Chèn dữ liệu vào bảng
        for index, row in df.iterrows():
            # Tránh lỗi cú pháp khi gặp dấu nháy đơn trong chuỗi
            row = row.apply(lambda x: str(x).replace("'", "''") if isinstance(x, str) else x)

            # Tạo lệnh INSERT
            columns = ', '.join([f"[{col}]" for col in df.columns])  # Đặt tên cột trong dấu ngoặc vuông
            placeholders = ', '.join(['?' for _ in df.columns])
            sql_query = f"INSERT INTO [{table_name}] ({columns}) VALUES ({placeholders})"  # Đặt tên bảng trong dấu ngoặc vuông
            
            # In câu lệnh SQL để kiểm tra
            print(f"Executing query: {sql_query}")
            cursor.execute(sql_query, tuple(row))
        
        sql_conn.commit()
        print(f"Đã lưu bảng '{table_name}' vào SQL Server.")
    except Exception as e:
        print(f"Lỗi khi lưu bảng '{table_name}' vào SQL Server: {e}")
    finally:
        cursor.close()

# Lấy dữ liệu từ PostgreSQL
def read_from_postgres(query, engine):
    try:
        return pd.read_sql(query, engine)
    except Exception as e:
        print(f"Lỗi khi đọc dữ liệu từ PostgreSQL: {e}")
        return None

# Thực thi di chuyển dữ liệu
def migrate_data():
    # Kết nối PostgreSQL và SQL Server
    pg_engine = connect_postgres(postgres_config)
    sql_server_conn = connect_sql_server(sql_server_config)

    # Các bảng cần di chuyển
    tables = [
        {"name": "projects", "query": "SELECT * FROM projects"},
        {"name": "locations", "query": "SELECT * FROM locations"},
        {"name": "project_details", "query": "SELECT * FROM project_details"},
        {"name": "project_types", "query": "SELECT * FROM project_types"},
        {"name": "status", "query": "SELECT * FROM status"}
    ]

    for table in tables:
        print(f"Đang xử lý bảng: {table['name']}")

        # Đọc dữ liệu từ PostgreSQL
        df = read_from_postgres(table["query"], pg_engine)
        if df is not None:
            # Lưu dữ liệu vào SQL Server
            write_to_sql_server(df, table["name"], sql_server_conn)

    # Đóng kết nối
    pg_engine.dispose()
    sql_server_conn.close()
    print("Di chuyển dữ liệu hoàn tất.")

# Gọi hàm di chuyển dữ liệu
migrate_data()


Đang xử lý bảng: projects
Bảng 'projects' đã được tạo hoặc đã tồn tại.
Executing query: INSERT INTO [projects] ([Mã loại dự án], [Tên dự án], [Mã dự án]) VALUES (?, ?, ?)
Executing query: INSERT INTO [projects] ([Mã loại dự án], [Tên dự án], [Mã dự án]) VALUES (?, ?, ?)
Executing query: INSERT INTO [projects] ([Mã loại dự án], [Tên dự án], [Mã dự án]) VALUES (?, ?, ?)
Executing query: INSERT INTO [projects] ([Mã loại dự án], [Tên dự án], [Mã dự án]) VALUES (?, ?, ?)
Executing query: INSERT INTO [projects] ([Mã loại dự án], [Tên dự án], [Mã dự án]) VALUES (?, ?, ?)
Executing query: INSERT INTO [projects] ([Mã loại dự án], [Tên dự án], [Mã dự án]) VALUES (?, ?, ?)
Executing query: INSERT INTO [projects] ([Mã loại dự án], [Tên dự án], [Mã dự án]) VALUES (?, ?, ?)
Executing query: INSERT INTO [projects] ([Mã loại dự án], [Tên dự án], [Mã dự án]) VALUES (?, ?, ?)
Executing query: INSERT INTO [projects] ([Mã loại dự án], [Tên dự án], [Mã dự án]) VALUES (?, ?, ?)
Executing query: INSERT INTO 

In [2]:
import pyodbc
import logging

# Cấu hình logging
logging.basicConfig(level=logging.INFO)

# Danh sách các lệnh SQL cho SQL Server
commands_sql_server = [
    'use cafeland;',  # Sử dụng cơ sở dữ liệu cafeland


    # Đảm bảo kiểu dữ liệu hợp lệ cho các cột trong các bảng
    'ALTER TABLE projects ALTER COLUMN [Mã dự án] INT NOT NULL;',
    'ALTER TABLE projects ALTER COLUMN [Mã loại dự án] INT NOT NULL;',
    'ALTER TABLE project_types ALTER COLUMN [Mã loại dự án] INT NOT NULL;',
    'ALTER TABLE locations ALTER COLUMN [Mã dự án] INT NOT NULL;',
    'ALTER TABLE project_details ALTER COLUMN [Mã dự án] INT NOT NULL;',
    'ALTER TABLE project_details ALTER COLUMN [Mã loại dự án] INT NOT NULL;',
    'ALTER TABLE status ALTER COLUMN [Mã dự án] INT NOT NULL;',
     
    
    # Khóa chính
    'ALTER TABLE projects ADD CONSTRAINT pk_project PRIMARY KEY ([Mã dự án]);',
    'ALTER TABLE project_types ADD CONSTRAINT pk_type PRIMARY KEY ([Mã loại dự án]);',

    # Khóa ngoại
    'ALTER TABLE locations ADD CONSTRAINT fk_locations_projects FOREIGN KEY ([Mã dự án]) REFERENCES projects ([Mã dự án]);',
    'ALTER TABLE project_details ADD CONSTRAINT fk_project_details_project FOREIGN KEY ([Mã dự án]) REFERENCES projects ([Mã dự án]);',
    'ALTER TABLE status ADD CONSTRAINT fk_status_project FOREIGN KEY ([Mã dự án]) REFERENCES projects ([Mã dự án]);',
    'ALTER TABLE projects ADD CONSTRAINT fk_type_project FOREIGN KEY ([Mã loại dự án]) REFERENCES project_types([Mã loại dự án]);'
]

def execute_sql_commands_sql_server(commands):
    try:
        # Kết nối SQL Server
        sql_server_conn = pyodbc.connect(
            r'DRIVER={ODBC Driver 17 for SQL Server};'
            r'SERVER=LAPTOP-OICT1D2U\SQLEXPRESS,1433;'
            r'DATABASE=cafeland;'
            r'UID=sa;'
            r'PWD=sapassword'
        )
        cursor = sql_server_conn.cursor()

        # Thực thi từng câu lệnh SQL
        for command in commands:
            logging.info(f"Executing query: {command}")
            cursor.execute(command)
            logging.info(f"Executed command: {command}")

        # Commit các thay đổi
        sql_server_conn.commit()

        # Kiểm tra số bản ghi bị ảnh hưởng
        rowcount = cursor.rowcount
        logging.info(f"{rowcount} rows affected.")

        # Đóng kết nối
        cursor.close()
        sql_server_conn.close()
        logging.info("SQL commands executed and committed successfully.")
    except Exception as e:
        logging.exception(f"Error executing SQL commands in SQL Server: {e}")

# Gọi hàm để thực thi
execute_sql_commands_sql_server(commands_sql_server)


INFO:root:Executing query: use cafeland;
INFO:root:Executed command: use cafeland;
INFO:root:Executing query: ALTER TABLE projects ALTER COLUMN [Mã dự án] INT NOT NULL;
INFO:root:Executed command: ALTER TABLE projects ALTER COLUMN [Mã dự án] INT NOT NULL;
INFO:root:Executing query: ALTER TABLE projects ALTER COLUMN [Mã loại dự án] INT NOT NULL;
INFO:root:Executed command: ALTER TABLE projects ALTER COLUMN [Mã loại dự án] INT NOT NULL;
INFO:root:Executing query: ALTER TABLE project_types ALTER COLUMN [Mã loại dự án] INT NOT NULL;
INFO:root:Executed command: ALTER TABLE project_types ALTER COLUMN [Mã loại dự án] INT NOT NULL;
INFO:root:Executing query: ALTER TABLE locations ALTER COLUMN [Mã dự án] INT NOT NULL;
INFO:root:Executed command: ALTER TABLE locations ALTER COLUMN [Mã dự án] INT NOT NULL;
INFO:root:Executing query: ALTER TABLE project_details ALTER COLUMN [Mã dự án] INT NOT NULL;
INFO:root:Executed command: ALTER TABLE project_details ALTER COLUMN [Mã dự án] INT NOT NULL;
INFO:r